In [1]:
import numpy as np
import graph
from graph import Graph
from collections import defaultdict

## Preparation Functions/Classes

In [2]:
def read_instance_file(filename):
    '''
    ::param: filename: string, filename that indicates the location of instance data file
    ::return value: (specification, data)
    :: specification: dict, specification of the instance
    :: data: the numpy array with a list of edges and their cost, demand
    :: data: [vertex1 vertex2 cost demand]
    '''
    fd = open(inputfile)
    content = fd.readlines()
    content = [x.strip() for x in content] 
    specification = dict()
    for i in range(8):
        line = content[i].split(':')
        specification[line[0].strip()] = line[1].strip()
    # print(specification)
    data = list()
    for line in content[9:-1]:
        tmp = line.split()
        data.append([int(x.strip()) for x in tmp])
    data = np.array(data)
    fd.close()
    return specification, data

In [3]:
'''
filelist CARP_samples
egl-e1-A.dat  gdb10.dat  val1A.dat  val7A.dat
egl-s1-A.dat  gdb1.dat   val4A.dat
'''
inputfile = 'CARP_samples/egl-s1-A.dat'

In [4]:
# reload graph.py
reload(graph)

<module 'graph' from 'graph.pyc'>

## Read the graph into Data Structure

In [5]:
spec, data = read_instance_file(inputfile)
gf = Graph()
gf.load_from_data(data.tolist())

In [6]:
print(gf)
print(spec)
print(gf.get_tasks())

vertices: 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47 48 49 50 51 52 53 54 55 56 57 58 59 60 61 62 63 64 65 66 67 68 69 70 71 72 73 74 75 76 77 78 79 80 81 82 83 84 85 86 87 88 89 90 91 92 93 94 95 96 97 98 99 100 101 102 103 104 105 106 107 108 109 110 111 112 113 114 115 116 117 118 119 120 121 122 123 124 125 126 127 128 129 130 131 132 133 134 135 136 137 138 139 140 
edges: set([1, 116]) set([2, 117]) set([128, 3]) set([4, 124]) set([5, 6]) set([8, 6]) set([6, 7]) set([8, 9]) set([8, 11]) set([9, 10]) set([10, 19]) set([10, 15]) set([33, 11]) set([11, 27]) set([11, 12]) set([12, 20]) set([12, 13]) set([16, 13]) set([17, 13]) set([13, 14]) set([16, 15]) set([17, 15]) set([17, 18]) set([18, 19]) set([24, 20]) set([20, 21]) set([20, 22]) set([21, 23]) set([24, 23]) set([26, 23]) set([24, 25]) set([25, 26]) set([25, 27]) set([26, 31]) set([27, 28]) set([28, 29]) set([28, 30]) set([33, 29]) set([32,

In [7]:
capacity = int(spec['CAPACITY'])
depot = int(spec['DEPOT'])

In [44]:
# test
print(gf.get_shortest_path(1,18))

([1, 116, 117, 119, 120, 121, 122, 87, 86, 85, 84, 82, 80, 79, 78, 77, 46, 43, 44, 45, 34, 139, 33, 11, 12, 13, 17, 18], 423)


In [9]:
gf[10]

{9: {'cost': 12}, 15: {'cost': 13}, 19: {'cost': 28}}

## Initialization
### Path-Scanning Simple

In [10]:
def which_better(u1, u2, graph, load, capacity, depot):
    import random
    r_cq1 = graph[u1[0]][u1[1]]['cost']/graph[u1[0]][u1[1]]['demand']
    r_cq2 = graph[u2[0]][u2[1]]['cost']/graph[u2[0]][u2[1]]['demand']
    return_cost1 = graph.get_shortest_path(u1[1], depot)[1]
    return_cost2 = graph.get_shortest_path(u2[1], depot)[1]
    # print(u1,u2)
    if load < capacity/2:
        if r_cq1 > r_cq2:
            return u1
        elif r_cq1 < r_cq2:
            return u2
        elif return_cost1 > return_cost2:
            return u1
        elif return_cost1 < return_cost2:
            return u2
        else:     
            return random.choice([u1, u2])
    else:
        if r_cq1 < r_cq2:
            return u1
        elif r_cq1 > r_cq2:
            return u2
        elif return_cost1 < return_cost2:
            return u1
        elif return_cost1 > return_cost2:
            return u2
        else:
            return random.choice([u1, u2])

def path_scanning(graph, depot):
    k = 0
    R = defaultdict(dict)
    load = defaultdict(dict)
    cost = defaultdict(dict)
    free_task = set(graph.get_tasks())
    while len(free_task) > 0:
        k += 1
        R[k] = list()
        load[k], cost[k] = 0, 0
        end = depot
        u = None
        #print("############ beign #############")
        while True:
            if len(free_task) == 0:
                break
            d_min = float('inf')
            for f_task in free_task:
#                 if f_task == (106,105):
#                     print(load[k], graph[f_task[0]][f_task[1]]['demand'])
                if graph[f_task[0]][f_task[1]]['demand'] + load[k] > capacity:
                    continue
                if u == None:
                    u = f_task
                    d_min = graph.get_shortest_path(end, f_task[0])[1]
                d_tmp = graph.get_shortest_path(end, f_task[0])[1]
                # print(d_tmp,d_min, end,f_task, u)
                if d_tmp < d_min:
                    d_min = d_tmp
                    u = f_task
                elif d_tmp == d_min:
                    d_min = d_tmp
                    #print(u)
                    u = which_better(u, f_task, gf, load[k], capacity, depot)
                    #print(u)
            if d_min == float('inf'):
                break
            #if u != None:
            R[k].append(u)
            free_task.remove(u)
            free_task.remove((u[1],u[0]))
            cost[k] += graph[u[0]][u[1]]['cost'] + d_min
            load[k] += graph[u[0]][u[1]]['demand']
            end = u[1]
        cost[k] += graph.get_shortest_path(u[1], depot)[1]
        #print("############ end #############")
    return R, load, cost

### Augment-merge

In [221]:
def concate_circles(graph, circle1, circle2, idx1, idx2):
    """
    params: circle: the path
    return [circle_list, overpapping_best, cost_largest]
    """
    overlapping_best = (0, 0)
    cost_largest = 0 # largest saving cost
    for p1, val1 in enumerate(circle1[-1:-1-idx1:-1]):
        for p2, val2 in enumerate(circle2[0:idx2]):
            if val1 == val2:
                cost_save_1 = Graph.calculate_path_cost(gf,circle1[-1:-1-p1-1])
                cost_save_2 = Graph.calculate_path_cost(gf,circle2[0:p2+1])
                save_total = cost_save_1 + cost_save_2
                if cost_largest<save_total:
                    overlapping_best = (p1,p2)
                    cost_largest = save_total
#     if cost_largest is 0:
#         print("gg", circle1, circle2, idx1, idx2)
    return circle1[:-1-overlapping_best[0]]+circle2[overlapping_best[1]:], overlapping_best, cost_largest

def get_first_last_req(circle):
    """
    params circle: dict
    """
    first, last = 0, 0
    for idx, val in enumerate(circle['circle'][-1::-1]):
        required = False
        for required_e in circle['aq_set']:
            if val in required_e:
                last = idx
                # print(val, required_e)
                required = True
                break
        if required:
            break
    for idx, val in enumerate(circle['circle']):
        required = False
        for required_e in circle['aq_set']:
            if val in required_e:
                first = idx
                required = True
                break
        if required:
            break
    return first,last

def merge_circles(graph, circle1, circle2):
    if circle1['load'] + circle2['load'] > capacity:
        return None
    idx1 = get_first_last_req(circle1)
    idx2 = get_first_last_req(circle2)
    # print(idx1,idx2)
    results = list()
    results.append(concate_circles(graph, circle1['circle'], circle2['circle'], idx1[1], idx2[0]))
    results.append(concate_circles(graph, circle1['circle'][::-1], circle2['circle'], idx1[0], idx2[0]))
    results.append(concate_circles(graph, circle2['circle'], circle1['circle'], idx2[1], idx1[0]))
    results.append(concate_circles(graph, circle2['circle'], circle1['circle'][::-1], idx2[1], idx1[1]))
    max_save = 0
    new_circle = None
    for result in results:
        if result[2]>=max_save:
            new_circle = result[0]
            max_save = result[2]
    if new_circle == None:
        # print(results)
        return None
    return {'circle':new_circle,'cost':circle1['cost']+circle2['cost']-max_save, 'saving': max_save, 
            'load':circle1['load'] + circle2['load'], 'aq_set':circle1['aq_set'].union(circle2['aq_set'])}

In [232]:
from itertools import tee, izip
def pairwise(iterable):
    "s -> (s0,s1), (s1,s2), (s2, s3), ..."
    a, b = tee(iterable)
    next(b, None)
    return izip(a, b)

def is_inclueded(path, target):
    inv_target =  (target[1], target[0])
    for edge in pairwise(path):
        # print(target, edge)
        if target == edge or inv_target == edge:
            return True
    return False
  


def augment_merge(graph, depot):
    # init routes
    k = 0
    free_tasks = set(graph.get_tasks_unique())
    R = defaultdict(dict)
    # cost = defaultdict(dict)
    for f_task in free_tasks:
        x = graph.get_shortest_path(depot, f_task[0])
        y = graph.get_shortest_path(f_task[1], depot)
        if x[1]==0:
            x = ([depot,],y[1])
        if y[1]==0:
            y[0] = ([depot,],x[1])
        R[k]['circle'] = x[0]+y[0]
        R[k]['cost'] = x[1]+y[1]
        R[k]['load'] = graph[f_task[0]][f_task[1]]['demand']
        R[k]['aq_set'] = set()
        R[k]['aq_set'].add(f_task)
        #print(f_task, R[k], cost[k])
        k += 1
    # augment phase
    R_sorted = sorted(R.iteritems(), key=lambda (k,v):len(v['circle']), reverse=True)
    deleted = set()
    for idx, circle in enumerate(R_sorted):
        for smaller_circle in R_sorted[idx+1:]:
            if smaller_circle[0] in deleted:
                continue
            if circle[1]['load'] + smaller_circle[1]['load'] > capacity:
                continue
            #print(smaller_circle[1][1]['aq_set'])
            flag_include = True
            for edge in smaller_circle[1]['aq_set']:
                #print(circle)
                if not is_inclueded(circle[1]['circle'],edge):
                    flag_include = False
            if flag_include:
                deleted.add(smaller_circle[0])
                circle[1]['aq_set'] = circle[1]['aq_set'].union(smaller_circle[1]['aq_set'])
                circle[1]['load'] += smaller_circle[1]['load']
                #print("[oK]", circle, smaller_circle)
    R_aug = list()
    for circle in R_sorted:
        if circle[0] not in deleted:
            circle[1]['delete'] = False
            R_aug.append(circle[1])
    
    # merge phase
    while True:
        merge_res = list()
        merge_next = list()
        for idx1, circle1 in enumerate(R_aug):
            for idx2, circle2 in enumerate(R_aug[idx1+1:]):
                #print(graph, circle1, circle2)
                tmp = merge_circles(graph, circle1, circle2)
                if tmp:
                    merge_res.append((tmp, idx1, idx1+idx2+1))
                    #print(tmp, idx1, idx2+idx1+1, circle1, circle2)
        if len(merge_res) is 0:
            break
        merge_res = sorted(merge_res, key=lambda (tmp,idx1,idx2):tmp['saving'], reverse=True)
        for merger in merge_res:
            if not R_aug[merger[1]]['delete'] and not R_aug[merger[2]]['delete']:
                # print(merger[1],merger[2])
                R_aug[merger[1]]['delete'] = True
                R_aug[merger[2]]['delete'] = True
                del(merger[0]['saving'])
                merger[0]['delete'] = False
                merge_next.append(merger[0])
        for circle in R_aug:
            if not circle['delete']:
                # print(circle)
                merge_next.append(circle)
        R_aug = merge_next
    R_final = R_aug
    return R_aug

In [237]:
init_solution1 = path_scanning(gf, depot)
init_solution2 = augment_merge(gf, depot)
print(init_solution1)
print(init_solution2)

(defaultdict(<type 'dict'>, {1: [(1, 116), (116, 117), (117, 119), (117, 2), (118, 114), (114, 113), (113, 112), (112, 110), (110, 107), (107, 112), (110, 111), (106, 105), (97, 98)], 2: [(107, 106), (105, 104), (104, 102), (66, 62), (62, 63), (63, 64), (64, 65), (56, 55), (55, 54), (55, 140), (140, 49), (49, 48), (139, 33), (12, 13)], 3: [(107, 108), (108, 109), (66, 67), (67, 68), (67, 69), (69, 71), (71, 72), (72, 73)], 4: [(87, 86), (86, 85), (85, 84), (84, 82), (82, 80), (80, 79), (79, 78), (78, 77), (77, 46), (46, 43), (43, 37), (37, 36), (36, 38), (38, 39), (39, 40)], 5: [(124, 126), (126, 130), (43, 44), (44, 45), (45, 34), (34, 139), (33, 11), (11, 12), (20, 22), (27, 28)], 6: [(95, 96), (96, 97), (73, 44), (11, 8), (8, 6), (6, 5), (8, 9), (13, 14), (24, 25)], 7: [(11, 27), (27, 25), (24, 20), (28, 30), (30, 32), (28, 29)]}), defaultdict(<type 'dict'>, {1: 210, 2: 210, 3: 207, 4: 208, 5: 209, 6: 210, 7: 140}), defaultdict(<type 'dict'>, {1: 738, 2: 910, 3: 778, 4: 642, 5: 1097

## Metaheuristics

## Solution-related

### output example

```
s 0,(1,2),(2,4),(4,1),0,0,(4,3),(3,1),0
q 15
```

In [250]:
float('inf')<1

False

In [332]:
x = [1,2,5,1,3]

In [338]:
x[2::-1]

[5, 2, 1]